# Tutorial

We now illustrate the basic capabilities of the ``respy`` package in a simple tutorial. The tutorial covers the following topics:

1. [How to specify a model - parameters and options.](#How-to-specify-a-model---parameters-and-options.)
2. [How to solve a model.](#How-to-solve-a-model.)
3. [How to simulate a data set.](#How-to-simulate-a-data-set.)
4. [How to estimate model parameters.](#How-to-estimate-model-parameters.)

After the [installation](installation.rst), the convention is to import ``respy`` as follows to get access to all exposed functions.

In [1]:
import respy as rp

## How to specify a model - parameters and options.

In order to define a model, ``respy`` relies on two objects - parameters and options. The difference between the two is that parameters are affected by the optimization whereas options stay the same. Here, only some parameters and options are explained. For a complete overview on parameters and options, visit the section [model specification](../software/model-specification.rst).

As an example, we turn to the first model of Keane and Wolpin (1994). The following function returns the parameters and options and the original dataset used by the authors.

In [2]:
params, options, data = rp.get_example_model("kw_94_one")

As said before, options are everything not affected by the optimization. The value under ``"n_periods"`` defines the number of periods in the model meaning the available time frame in which individual can make their decisions.

Choices which are available to individuals within the model are specified under ``"choices"``. An empty dictionary as a value signals that this choice relies on default values generated inside of ``respy`` or does not need information at all. Under ``"edu"`` we can see which options are available for each choice.

- ``"max"`` defines the maximum amount of experience which can be accumulated by individuals using this choice.
- ``"start"`` is a list and contains different initial experience levels of this choice. Here, individuals start with 10 years of experience.
- ``"lagged"`` defines the share of individual with this choice and initial experience level having ``"edu"`` as the initial lagged choice when entering the model.
- ``"share"`` influences the probability of being a particular type.

The rest of the options includes seeds, numbers of draws to simulate the flow utilities in each period and other model parameters. Furthermore, there is a lot of logic hidden from the user to offer a comprehensible starting point. For a deeper understanding of the mechanics, the user is refered to the section [model specfication](../software/model-specification.rst).

In [3]:
options['estimation_tau'] = 100
options['estimation_draws'] = 2000

The parameter specification includes all parameters of the model which are affected by the optimization routine.

In [4]:
params.head()

value  fixed  lower  upper  \
category name                                       
delta    delta         0.950  False    0.7    1.0   
wage_a   constant      9.210  False    NaN    NaN   
         exp_edu       0.038  False    NaN    NaN   
         exp_a         0.033  False    NaN    NaN   
         exp_a_square -0.050  False    NaN    NaN   

                                                                 comment  
category name                                                             
delta    delta                                           discount factor  
wage_a   constant      skill rental price if the base skill endowment...  
         exp_edu       linear return to an additional year of schooli...  
         exp_a          return to experience, same sector, linear (wage)  
         exp_a_square  return to experience, same sector, quadratic (...

The dataset is already in the correct format for estimation.

In [5]:
data.head()

,Identifier,Period,Choice,Wage,Experience_A,Experience_B,Experience_Edu,Lagged_Choice
0,0,0,edu,NaN,0,0,11,NaN
1,0,1,edu,NaN,0,0,12,edu
2,0,2,edu,NaN,0,0,13,edu
3,0,3,edu,NaN,0,0,14,edu
4,0,4,edu,NaN,0,0,15,edu


## How to solve a model.

The solution of the model is the value of each state within the model assuming optimal decision in this and future periods. In ``respy``, the solution is represented with the :class:`~respy.state_space.StateSpace`, a class including the model solution. The solution combines the following steps:

- Building the state space.
- Solving the model by finding optimal decision in each state via backward induction.

The ``state_space`` is normally hidden from the user and only an internal construct. For a better understanding of how modeling works, the state space is exposed to the user.

In [6]:
state_space = rp.solve(params, options)

## How to simulate a data set.

Simulating a data set is as easy as passing model inputs to :func:`~respy.simulate.simulate`. The return values are the ``state_space`` of the model and ``df``, a data set of simulated agents as a :class:`pandas.DataFrame`. The number of simulated agents can be set in the options under ``"simulation_agents"``.

In [7]:
state_space, df = rp.simulate(params, options)

## How to estimate model parameters.

To estimate model parameters via maximum likelihood, ``respy`` relies on [``estimagic``](https://github.com/OpenSourceEconomics/estimagic), an open-source tool to estimate structural models and more. First, we need the criterion function of the model which returns the likelihood value of the data given the model.

In [8]:
crit_func = rp.get_crit_func(params, options, df)

The criterion function takes the parameters as an input and returns the likelihood value.

In [9]:
crit_val = crit_func(params)
crit_val

/home/janos/Dropbox/Projects/respy/respy/conditional_draws.py:80: RuntimeWarning: overflow encountered in exp
  draws[:, :n_wages] = np.clip(np.exp(draws[:, :n_wages]), 0.0, HUGE_FLOAT)


-431.08970501343833

To estimate the model parameters, import the :func:`~estimagic.optimization.optimize.minimize` from ``estimagic``.

In [10]:
import numpy as np
from estimagic.optimization.optimize import maximize

For ``estimagic``, we need to pass constraints on the parameters in a list containing dictionaries. Each dictionary is a constraint. A constraint includes two components: First, we need to tell ``estimagic`` which paramters we want to constrain. This is achieved by specifying an index location which will be passed to `df.loc`. Then, define the type of the constraint. Here, we only impose the constraint that the shock parameters have to be valid variances and correlations.

In [11]:
constr = [{"loc": "shocks", "type": "sdcorr"}, {"loc": "delta", "type": "fixed", "value": 0.95}]

Optionally, we can add a column ``"group"`` which is identical to the category column. The estimagic dashboard will then contain one parameter convergence plot per group instead of plotting all parameters in the same figure. Since respy has quite many parameters, this will make the plots much more readable.

In [12]:
params["group"] = params.index.get_level_values('category')
params['lower'].fillna(-np.inf, inplace=True)
params['upper'].fillna(np.inf, inplace=True)
params

value  fixed  lower  upper  \
category    name                                                        
delta       delta                          0.950  False    0.7    1.0   
wage_a      constant                       9.210  False   -inf    inf   
            exp_edu                        0.038  False   -inf    inf   
            exp_a                          0.033  False   -inf    inf   
            exp_a_square                  -0.050  False   -inf    inf   
            exp_b                          0.000  False   -inf    inf   
            exp_b_square                   0.000  False   -inf    inf   
            hs_graduate                    0.000  False   -inf    inf   
            co_graduate                    0.000  False   -inf    inf   
            period                         0.000  False   -inf    inf   
            is_minor                       0.000  False   -inf    inf   
            any_exp_a                      0.000  False   -inf    inf   
            work_a_lagged                  0.000  False   -inf    inf   
nonpec_a    constant                       0.000  False   -inf    inf   
            not_exp_a_lagged               0.000  False   -inf    inf   
            not_any_exp_a                  0.000  False   -inf    inf   
            hs_graduate                    0.000  False   -inf    inf   
            co_graduate                    0.000  False   -inf    inf   
wage_b      constant                       8.480  False   -inf    inf   
            exp_edu                        0.070  False   -inf    inf   
            exp_a                          0.022  False   -inf    inf   
            exp_a_square                  -0.050  False   -inf    inf   
            exp_b                          0.067  False   -inf    inf   
            exp_b_square                  -0.100  False   -inf    inf   
            hs_graduate                    0.000  False   -inf    inf   
            co_graduate                    0.000  False   -inf    inf   
            period                         0.000  False   -inf    inf   
            is_minor                       0.000  False   -inf    inf   
            any_exp_b                      0.000  False   -inf    inf   
            work_b_lagged                  0.000  False   -inf    inf   
nonpec_b    constant                       0.000  False   -inf    inf   
            not_exp_b_lagged               0.000  False   -inf    inf   
            not_any_exp_b                  0.000  False   -inf    inf   
            hs_graduate                    0.000  False   -inf    inf   
            co_graduate                    0.000  False   -inf    inf   
nonpec_edu  constant                       0.000  False   -inf    inf   
            is_return_not_high_school  -4000.000  False   -inf    inf   
            is_return_high_school      -4000.000  False   -inf    inf   
            period                         0.000  False   -inf    inf   
            is_minor                       0.000  False   -inf    inf   
            hs_graduate                    0.000  False   -inf    inf   
            co_graduate                    0.000  False   -inf    inf   
nonpec_home constant                   17750.000  False   -inf    inf   
            is_young_adult                 0.000  False   -inf    inf   
            is_adult                       0.000  False   -inf    inf   
            hs_graduate                    0.000  False   -inf    inf   
            co_graduate                    0.000  False   -inf    inf   
shocks      sd_a                           0.200  False   -inf    inf   
            sd_b                           0.250  False   -inf    inf   
            sd_edu                      1500.000  False   -inf    inf   
            sd_home                     1500.000  False   -inf    inf   
            corr_b_a                       0.000  False   -inf    inf   
            corr_edu_a                     0.000  False   -inf    inf   
            corr_edu_b      

In [ ]:
maximize(crit_func, params, "nlopt_newuoa_bound", db_options={"rollover": 200}, constraints=constr, dashboard=True)

/home/janos/Dropbox/Projects/respy/respy/conditional_draws.py:80: RuntimeWarning: overflow encountered in exp
  draws[:, :n_wages] = np.clip(np.exp(draws[:, :n_wages]), 0.0, HUGE_FLOAT)


Bokeh app running at: http://localhost:36689/


/home/janos/Dropbox/Projects/respy/respy/conditional_draws.py:80: RuntimeWarning: overflow encountered in exp
  draws[:, :n_wages] = np.clip(np.exp(draws[:, :n_wages]), 0.0, HUGE_FLOAT)
/home/janos/Dropbox/Projects/respy/respy/likelihood.py:299: RuntimeWarning: divide by zero encountered in log
  contribs = np.clip(np.log(contribs), -HUGE_FLOAT, HUGE_FLOAT)


## References

> Keane, M. P. and  Wolpin, K. I. (1994b). [The Solution and Estimation of Discrete Choice Dynamic Programming Models by Simulation and Interpolation: Monte Carlo Evidence](https://www.minneapolisfed.org/research/staff-reports/the-solution-and-estimation-of-discrete-choice-dynamic-programming-models-by-simulation-and-interpolation-monte-carlo-evidence). *Federal Reserve Bank of Minneapolis*, No. 181.